In [1]:
import json
from glob import glob

In [2]:
problems = json.load(open('./problems_with_builtin_answers.json'))

In [3]:
print(sum(1 for obj in problems if obj['solution']), 'builtin answers given by website')
print(len(problems), 'questions in total')

47 builtin answers given by website
317 questions in total


In [4]:
# inspect ozelentok's answers
def collect_signatures(lines):
    return [line.strip() for line in lines if line.startswith('public')]
signatures = []
for fn in glob('./solutions/ozelentok/*.java'):
    signatures += collect_signatures([line for line in open(fn)])
print(len(signatures), 'total solutions;', len(set(signatures)), 'unique signatures')
print(sum(1 for prob in problems if prob['signature'].strip() in signatures), 'solutions with matching signatures')

254 total solutions; 252 unique signatures
246 solutions with matching signatures


In [5]:
# extract ozelentok's code
def map_signatures(filenames):
    ret = {}
    
    last_signature = None
    for fn in filenames:
        print(fn)
        with open(fn) as f:
            for line in f:
                if line.lstrip().startswith('public'):
                    last_signature = line.strip()
                    ret[last_signature] = [last_signature + '{']
                elif line.startswith('{'):
                    # ret[last_signature] = []
                    pass
                elif line.startswith('}'):
                    ret[last_signature].append('}')
                    ret[last_signature] = '\n'.join(ret[last_signature])
                    last_signature = None
                elif last_signature is not None:
                    ret[last_signature].append(line.strip())
    return ret
oz_code = map_signatures(glob('./solutions/ozelentok/*.java'))

./solutions/ozelentok/Recursion-1.java
./solutions/ozelentok/Array-3.java
./solutions/ozelentok/Recursion-2.java
./solutions/ozelentok/String-3.java
./solutions/ozelentok/Array-2.java
./solutions/ozelentok/String-1.java
./solutions/ozelentok/Logic-1.java
./solutions/ozelentok/Warmup-2.java
./solutions/ozelentok/Logic-2.java
./solutions/ozelentok/Warmup-1.java
./solutions/ozelentok/Array-1.java
./solutions/ozelentok/String-2.java


In [6]:
# inspect mirandaoi's answers
print(len(glob('./solutions/mirandaio/*/*.java')), 'unique solutions')

280 unique solutions


In [7]:
# extract mirandoi's code
def map_signatures2(filenames):
    ret = {}
    
    for fn in filenames:
        category, problem = fn.split('/')[-2:]
        problem = problem[:-len('.java')] # strip file extension
        key = (category.lower(), problem.lower())
        with open(fn) as f:
            ret[key] = ''.join(f)   
    return ret
mi_code = map_signatures2(glob('./solutions/mirandaio/*/*.java'))

In [8]:
# extract sergey's answers
def map_signatures3(filenames):
    ret = {}
    
    last_signature = None
    for fn in filenames:
        with open(fn) as f:
            for line in f:
                if line.lstrip().startswith('public static'):
                    last_signature = line.strip().replace('static ', '')
                    ret[last_signature] = [last_signature + '{']
                elif line.startswith('    }') and last_signature:
                    ret[last_signature].append('}')
                    ret[last_signature] = '\n'.join(ret[last_signature])
                    last_signature = None
                elif last_signature is not None:
                    ret[last_signature].append(line.strip())
    return ret
sk_code = map_signatures3(glob('./solutions/SergeyKandalintsev/src/*/*.java'))

In [9]:
# join answers to the appropriate list
list(sk_code.keys())[:3], list(mi_code.keys())[:3], list(oz_code.keys())[:3]

(['public boolean scoresClump( int[] scores )',
  'public String without2(String str) {',
  'public String startWord(String str, String word) {'],
 [('warmup-1', 'intmax'),
  ('string-1', 'withoutend2'),
  ('warmup-1', 'starthi')],
 ['public int[] fix45(int[] nums)',
  'public int maxBlock(String str)',
  'public String noX(String str)'])

In [10]:
for obj in problems:
    sk_key = obj['title'] + '('
    if any(sk_key in key for key in sk_code.keys()):
        for k, v in sk_code.items():
            if sk_key in k:
                obj['solution'].append(v)
                break
    
    if obj['signature'] in oz_code:
        obj['solution'].append(oz_code[obj['signature']])
    
    mi_key = (obj['section'].lower(), obj['title'].lower())
    if mi_key in mi_code:
        obj['solution'].append(mi_code[mi_key])

In [11]:
# get prelim stats on answers
from collections import Counter
print(Counter(len(o['solution']) for o in problems))

Counter({2: 161, 3: 67, 0: 37, 4: 32, 1: 20})


In [12]:
json.dump(problems, open('./problems_with_answers.json', 'w'))